# 一、图的定义和分类
1、图与子图  
<img src="../pics/23.png" width="50%" />  
2、有无向、有无环  
<img src="../pics/24.png" width="50%" />  
# 二、图的存储结构和问题应用  
0、顶点数：n   边数：m  顶点$v_i$的度数：$TD(v_i)$  
1、邻接矩阵（带权）  
<img src="../pics/25.png" width="50%" />  
2、邻接表：一种结合顺序存储与链式存储的图结构。它主要由两部分组成：一是用于存放所有顶点信息的数组，二是用于存放每个顶点所有邻接边的链表。  
<img src="../pics/26.png" width="50%" />  

In [ ]:
# 邻接矩阵的代码实现
class Graph:
    def __init__(self,ver_count,directed=False,inf=float("inf")):
        self.n=ver_count # 顶点数n
        self.directed=directed # 是否为有向图
        self.inf=inf # 无边时填充值
        self.adj = [[inf]*(ver_count+1) for _ in range(ver_count+1)] # 邻接矩阵（0号行列弃用，直观）
        for i in range(1,ver_count+1):
            self.adj[i][i]=0

    def add_edge(self,vi,vj,w=1):
        self.adj[vi][vj]=w
        if not self.directed:
            self.adj[vj][vi]=w
    
    def get_edge(self,vi,vj):
        if self.adj[vi][vj] != self.inf:
            return self.adj[vi][vj]
        return None
    
    def printMatrix(self):
        for i in range(1,self.n+1):
            row=[self.adj[i][j] if self.adj[i][j] != self.inf else "∞" for j in range(1,self.n+1)]
            print( " ".join(map(str,row)))

graph = Graph(6, directed=True)
edges = [(1, 2, 5), (1, 5, 6), (2, 4, 7), (4, 3, 9), (3, 1, 2), (5, 6, 8), (6, 4, 3)]
for u,v,w in edges:
    graph.add_edge(u,v,w)

print(graph.get_edge(4, 3)) # 输出 9（存在边 4->3，权重 9）
print(graph.get_edge(4, 5)) # 输出 None（不存在边 4->5）
graph.printMatrix()

9
None
0 5 ∞ ∞ 6 ∞
∞ 0 ∞ 7 ∞ ∞
2 ∞ 0 ∞ ∞ ∞
∞ ∞ 9 0 ∞ ∞
∞ ∞ ∞ ∞ 0 8
∞ ∞ ∞ 3 ∞ 0


In [5]:
# 邻接表的代码实现  
class EdgeNode:
    # 边结点
    def __init__(self,vj,val):
        self.vj=vj
        self.val=val
        self.next=None # 指向下一条同起点的边

class VertexNode:
    # 顶点结点：存储顶点编号与其第一条邻接边
    def __init__(self,vi):
        self.vi=vi
        self.head=None

class Graph:
    def __init__(self,ver_count,directed=False):
        self.n = ver_count
        self.directed = directed
        # 使用1..n的顶点编号，0号位置空置，便于直观
        self.vertices = [None] + [VertexNode(i)for i in range(1,ver_count +1)]

    def _valid(self,v): # 顶点合法性检查
        return 1<=v<=self.n
    
    def add_edge(self,vi,vj,val=1):
        if not self._valid(vi) or not self._valid(vj):
            raise ValueError("invalid vertex:{} or {}".format(vi,vj))
        edge=EdgeNode(vj,val)
        # 用头插法，后来的用来当第一个
        edge.next=self.vertices[vi].head # 指向现在的第一名
        self.vertices[vi].head=edge
        if not self.directed:
            rev=EdgeNode(vi,val)
            rev.next=self.vertices[vj].head
            self.vertices[vj].head=rev

    def get_edge(self,vi,vj):
        if not self._valid(vi) or not self._valid(vj):
            raise ValueError("invalid vertex:{} or {}".format(vi,vj))
        cur = self.vertices[vi].head
        while cur:
            if cur.vj == vj:
                return cur.val
            cur=cur.next
        return None
    
    def neighbors(self,vi):
        cur=self.vertices[vi].head
        while cur:
            yield cur.vj,cur.val
            cur=cur.next
    
    def printGraph(self):
        for vi in range(1,self.n+1):
            cur=self.vertices[vi].head
            while cur:
                print(str(vi)+" - "+str(cur.vj)+" : "+str(cur.val))
                cur=cur.next
#示例：构建有向带权图并查询/打印
graph =Graph(6,directed=True)
edges =[(1,2,5),(1,5, 6),(2,4, 7),(4,3,9),(3,1,2),(5,6,8), (6,4,3)]
for u, v, w in edges:
    graph.add_edge(u,v,w)
print(graph.get_edge(4,3)) #9
print(graph.get_edge(4,5)) #None（无此边）
graph.printGraph()

9
None
1 - 5 : 6
1 - 2 : 5
2 - 4 : 7
3 - 1 : 2
4 - 3 : 9
5 - 6 : 8
6 - 4 : 3


3、链式前向星：静态邻接表，是一种以静态链表实现邻接表的高效图存储结构。  
（1）边集数组edges：edges[i]表示第i条边，包含信息.vj为该边终点，.val为该边权值，.next指向与该边起点相同的下一条边在edges数组中的下标。  
（2）头节点数组head：head[i]存储以顶点i为起点的第一条边在edges数组中的下标  
<img src="../pics/27.png" width="50%" />  
4、哈希表实现邻接表：使用一个字典存储所有顶点信息，字典的键为顶点编号，值为该顶点的邻接边集合（同样用一个字典表示）

In [1]:
n=6
edges=[(1,2,5),(1,5, 6),(2,4, 7),(4,3,9),(3,1,2),(5,6,8), (6,4,3)]
graph={i:{} for i in range(1,n+1)}
for vi,vj,val in edges:
    graph[vi][vj]=val
for vi in range(1, n + 1):
    print(f"顶点 {vi} 的邻接点: {list(graph[vi].keys())}")

顶点 1 的邻接点: [2, 5]
顶点 2 的邻接点: [4]
顶点 3 的邻接点: [1]
顶点 4 的邻接点: [3]
顶点 5 的邻接点: [6]
顶点 6 的邻接点: [4]


# 三、深度优先搜索
1、深度优先搜索算法：核心思想是「沿一条路径尽可能深入」，遇到无法继续的节点时再回溯到上一个分叉点，继续探索其他路径，直到遍历完整个结构或找到目标为止。  
<img src="../pics/28.png" width="75%" />  

In [7]:
class Solution:
    # 递归实现深度优先搜索（DFS）
    def dfs_recursive(self,graph,u, visited):
        print(u) # 当前访问节点
        visited.add(u)
        for v in graph[u]:
            if v not in visited:
                self.dfs_recursive(graph,v,visited)
    # 堆栈实现DFS
    def dfs_stack(self, graph,u):
        visited = set()
        stack=[]
        stack.append([u,0]) # u:起始节点 0：下一个访问邻接节点的下标
        visited.add(u)
        print(u)

        while stack:
            cur,idx=stack.pop()
            neighbors = graph[cur]
            if idx < len(neighbors): # 没有遍历完
                v = neighbors[idx]
                stack.append([cur,idx+1]) 

                if v not in visited:
                    print(v)
                    visited.add(v)
                    stack.append([v,0])
        return visited

               


 ##示例图（邻接表形式，节点为字符串，边为无向边）
graph ={
"1":["2","3"],
"2":["1","3","4"],
"3":["1","2","4","5"],
"4":["2","3","5","6"],
"5":["3","4"],
"6":["4","7"],
"7":["6"]
}

# 初始化已访问节点集合
visited = set()
# 从节点"1"开始进行深度优先搜索
#Solution().dfs_recursive(graph,"1",visited)
Solution().dfs_stack(graph, "1")

1
2
3
4
5
6
7


{'1', '2', '3', '4', '5', '6', '7'}

# 四、广度优先搜索
1、广度优先搜索算法（BFS）：从起始节点出发，按照层级逐步向外扩展，优先访问距离起点较近的节点，再访问距离较远的节点，直到遍历完整个结构或找到目标节点。  
<img src="../pics/29.png" width="75%" />  

In [9]:
import collections

class Solution():
    # 队列实现的BFS
    def bfs(self,graph,u):
        visited=set()
        queue=collections.deque()

        visited.add(u)
        queue.append(u)
        while queue:
            u=queue.popleft()
            print(u)
            for v in graph[u]:
                if v not in visited:
                    visited.add(v)
                    queue.append(v)

graph ={
"1":["2","3"],
"2":["1","3","4"],
"3":["1","2","4","5"],
"4":["2","3","5","6"],
"5":["3","4"],
"6":["4","7"],
"7":["6"]
}
Solution().bfs(graph,"1")


1
2
3
4
5
6
7
